In [1]:
from pynq.overlays.base import BaseOverlay
from IPython.display import clear_output
from pynq import allocate
from time import sleep
import plotly.graph_objs as go
import ipywidgets as ipw
import numpy as np
import threading
import socket
import sys
import xrfdc

overlay = BaseOverlay('base.bit')

UDP_IP = "10.42.0.1" #PC-10.42.0.1 (enx0050b623bd36) RFSoC-10.42.0.85 General-127.0.0.1
UDP_PORT = 5005

In [2]:
for led in overlay.leds:
    led.off()

overlay.init_rf_clks()

#---Socket---
sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM)  # UDP

#---DAC & ADC---
freq = 500

num_dac = 1 #DAC_A
dac = overlay.radio.transmitter.channel[num_dac]
dac.dac_block.MixerSettings['Freq'] = freq # MHz 
dac.control.gain = 0.001
dac.control.enable = True
    
num_adc = 3 #ADC_A
adc = overlay.radio.receiver.channel[num_adc]    
adc.adc_tile.DynamicPLLConfig(1, 491.52, 4915.2)  #f_ADC = 4915.2MHz  f_sampling = fADC/decimation_factor
                                                #def DynamicPLLConfig(self, source, ref_clk_freq, samp_rate):
                                                #Si cambio ref_clk_freq:  Input reference clock frequency (91.520000 MHz) does not respect the specifications for internal PLL usage. Please use a different frequency (102.406250 - 614.000000 MHz) or bypass the internal PLL for ADC 2 in XRFdc_DynamicPLLConfig
adc.adc_block.DecimationFactor = 40 #1x (bypass), 2x, 3x, 4x, 5x, 6x, 8x, 10x, 12x, 16x, 20x, 24x, 40x al final sera DecimationFactor multiplicado por x15
sleep(0.6)
adc.adc_block.NyquistZone = 1
adc.adc_block.MixerSettings = {
    'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,   #Cambiarlo no hace nada
    'EventSource':    xrfdc.EVNT_SRC_TILE,       #Cambiarlo no hace nada
    'FineMixerScale': xrfdc.MIXER_SCALE_1P0,     #Cambiarlo no hace nada
    'Freq':           -freq,                 #F_mixer
    'MixerMode':      xrfdc.MIXER_MODE_R2C, #MIXER_MODE_C2C parecia que no perdia tantas muestras
    'MixerType':      xrfdc.MIXER_TYPE_FINE, #MIXER_TYPE_COARSE fa que 'Freq'=0 sempre
    'PhaseOffset':    0.0                      #Cambiarlo no hace nada
}
adc.adc_block.UpdateEvent(xrfdc.EVENT_MIXER) #importante no borrar
adc.adc_tile.SetupFIFO(True)





power = overlay.switches.read()+4;
number_samples = 2**power

import ipywidgets as ipw
from rfsystem.spectrum_sweep import SpectrumAnalyser

analysers = []
sample_frequency = 4915.2/adc.adc_block.DecimationFactor
analysers.append(
    SpectrumAnalyser(channel = adc,
                     sample_frequency = sample_frequency,
                     number_samples = number_samples,
                     title = ''.join(['Spectrum Analyser Channel RX']),
                     height = None,
                     width = None))

ipw.VBox([analyser.spectrum_plot.get_plot() for analyser in analysers])

    'data': [{'name': 'Spectrum',
              'type': 'scatter',
             …

In [3]:
for led in overlay.leds:
    led.on()
    
power = overlay.switches.read()+4;
number_samples = 2**power

#---Send UPD---
if (power > 15):
    print("Packet size incorrect, power value should be in range 4 to 15.")
else:
    while(overlay.buttons[0].read()==0):
        sock.sendto(np.array(adc.transferIQ(number_samples)), (UDP_IP, UDP_PORT)) 
                    #np.array per passar de list a array
        for analyser in analysers:
            analyser.update_spectrum()

print('End of connection...')
for led in overlay.leds:
    led.off()

KeyboardInterrupt: 

In [ ]:
#Si error OSError: [Errno 90] Message too long:
#
    #You can do that by changing the MTU (Maximum Transfer Unit) of your interface. 
    #This is the maximal number of bytes a frame can contain.
    #
    #ifconfig eth0 mtu 2000 up
    
#Ara per ara (20-10-23), es queixa quan són més de "Power value: 13"